<a href="https://colab.research.google.com/github/pawljmlo/covid-ur-wasting/blob/main/plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>